In [1]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 1.1 MB/s eta 0:00:002.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 439.7 kB/s eta 0:00:00 MB/s eta 0:00:01


In [48]:
from geopy.geocoders import Nominatim # https://nominatim.org/
import numpy as np
import pandas as pd
import random
import time

In [9]:
df = pd.read_csv('./NJOP2021.csv')
df = df.drop(['Unnamed: 0', 'blk', 'znt', 'kelas_bumi', 'sd', 'page_num'], axis=1)
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,min_njb,max_njb,ave_njb
0,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,GG FLAMBOYAN,"5,625,000","5,900,000","5,763,000"
1,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,GG FLAMBOYAN,"5,095,000","5,350,000","5,223,000"
2,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,GG H MASRIKI,"5,625,000","5,900,000","5,763,000"
3,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,GG H SAEMIN,"5,625,000","5,900,000","5,763,000"
4,: 31 - DKI JAKARTA,: 71 - JAKARTA SELATAN,: 010 - KEBAYORAN LAMA,: 001 - PONDOK PINANG,GG H SAEMIN,"5,095,000","5,350,000","5,223,000"


In [10]:
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df[col] = df[col].apply(lambda x: str(x).split(' - ')[1])

In [12]:
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,min_njb,max_njb,ave_njb
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG FLAMBOYAN,"5,625,000","5,900,000","5,763,000"
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG FLAMBOYAN,"5,095,000","5,350,000","5,223,000"
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H MASRIKI,"5,625,000","5,900,000","5,763,000"
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAEMIN,"5,625,000","5,900,000","5,763,000"
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAEMIN,"5,095,000","5,350,000","5,223,000"


In [26]:
# test Nominatim performance with kelurahan and kecamatan and see if there is difference
# use sample size of 300
parsed_street = []
parsed_street_kel = []
parsed_street_kel_kec = []
random.seed(42)
for n in random.sample(range(0, len(df)), 300):
    print('Now on sample', n)
    geolocator = Nominatim(user_agent="yusuf-application")
    location = geolocator.geocode(df.iloc[n]['nama_jalan'], timeout=10)
    if location:
        parsed_street.append((location.latitude, location.longitude))
    geolocator = Nominatim(user_agent="yusuf-application")
    location = geolocator.geocode(df.iloc[n]['nama_jalan']+', '+df.iloc[n]['kelurahan'], timeout=10)
    if location:
        parsed_street_kel.append((location.latitude, location.longitude))
    geolocator = Nominatim(user_agent="yusuf-application")
    location = geolocator.geocode(df.iloc[n]['nama_jalan']+', '+df.iloc[n]['kelurahan']+', '+df.iloc[n]['kecamatan'], timeout=10)
    if location:
        parsed_street_kel_kec.append((location.latitude, location.longitude))

Now on sample 167621
Now on sample 29184
Now on sample 6556
Now on sample 194393
Now on sample 72097
Now on sample 64196
Now on sample 58513
Now on sample 36579
Now on sample 193061
Now on sample 26868
Now on sample 177392
Now on sample 194161
Now on sample 142964
Now on sample 22790
Now on sample 154794
Now on sample 110604
Now on sample 8331
Now on sample 7811
Now on sample 24561
Now on sample 57314
Now on sample 60990
Now on sample 132475
Now on sample 157815
Now on sample 6956
Now on sample 147127
Now on sample 52124
Now on sample 187700
Now on sample 170363
Now on sample 183848
Now on sample 142853
Now on sample 109974
Now on sample 57787
Now on sample 117757
Now on sample 154472
Now on sample 72926
Now on sample 212187
Now on sample 1703
Now on sample 198916
Now on sample 211240
Now on sample 41853
Now on sample 183013
Now on sample 110785
Now on sample 89194
Now on sample 72842
Now on sample 40758
Now on sample 56443
Now on sample 200145
Now on sample 88236
Now on sample 26793
N

In [28]:
print('Number of parsed street:', len(parsed_street))
print('Number of parsed street with kelurahan:', len(parsed_street_kel))
print('Number of parsed street with kelurahan and kecamatan:', len(parsed_street_kel_kec))

Number of parsed street: 142
Number of parsed street with kelurahan: 106
Number of parsed street with kelurahan and kecamatan: 92


In [31]:
print(parsed_street)
print(parsed_street_kel)
print(parsed_street_kel_kec)

[(-0.2406528, 115.7893091), (-6.3591693, 106.9090969), (-7.96107765, 112.64374034779982), (-6.1483712, 106.7837231), (-6.2210997, 106.7950565), (-6.3102599, 106.9585946), (-6.189704, 106.8076511), (-6.2160351, 106.7674904), (-6.2128386, 106.7864603), (-6.8923918, 107.6261032), (-6.153472, 106.8206711), (-6.3330353, 106.8528784), (1.1416294, 104.0135267), (3.1561927, 101.67605422877257), (-6.1985579, 106.9921284), (-6.2552123, 106.8315542), (-6.219273, 106.9049734), (-6.2617799, 106.8364218), (-10.0438735, 123.9934271), (-6.2872405, 106.8305449), (1.1474136499999998, 104.00507859999999), (-6.2861829, 106.8258963), (1.1475412, 104.0285752), (-6.2896723, 106.8296277), (-6.3896555, 106.8371331), (1.0503782, 103.9822641), (-6.2336546, 106.8641206), (-6.1634339, 106.7443807), (-6.3896555, 106.8371331), (-5.993584, 120.4819367), (-6.1097212, 106.92519682261829), (-6.3297998, 106.8584229), (-6.9518903, 109.3270782), (-8.6377472, 115.2674898), (2.3490733, 102.1093729), (-6.2899084, 106.9651559)

Now this is weird:
1. If we only parse the street, we may get inaccurate result, i.e., some coordinates are not even in Jakarta
1. If we include more parameters, the number of result decreases

Since data quality is our main focus, we will go with street name and kelurahan

In [45]:
df = pd.read_csv('./NJOP2021.csv')
df = df[['provinsi', 'kota', 'kecamatan', 'kelurahan', 'nama_jalan']]
cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df[col] = df[col].apply(lambda x: str(x).split(' - ')[1])
df = df.drop_duplicates(ignore_index=True)
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG FLAMBOYAN
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H MASRIKI
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAEMIN
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAIMIN
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H. SAIMIN


In [49]:
dict = {}
for idx in range(len(df)):
    print('Now on index', idx)
    try:
        geolocator = Nominatim(user_agent="yusuf-application")
        location = geolocator.geocode(df.iloc[idx]['nama_jalan']+', '+df.iloc[idx]['kelurahan'], timeout=10)
        if location:
            dict[idx] = [location.latitude, location.longitude]
    except:
        idx -= 1
        time.sleep(60) # wait for internet to reconnect

Now on index 0
Now on index 1
Now on index 2
Now on index 3
Now on index 4
Now on index 5
Now on index 6
Now on index 7
Now on index 8
Now on index 9
Now on index 10
Now on index 11
Now on index 12
Now on index 13
Now on index 14
Now on index 15
Now on index 16
Now on index 17
Now on index 18
Now on index 19
Now on index 20
Now on index 21
Now on index 22
Now on index 23
Now on index 24
Now on index 25
Now on index 26
Now on index 27
Now on index 28
Now on index 29
Now on index 30
Now on index 31
Now on index 32
Now on index 33
Now on index 34
Now on index 35
Now on index 36
Now on index 37
Now on index 38
Now on index 39
Now on index 40
Now on index 41
Now on index 42
Now on index 43
Now on index 44
Now on index 45
Now on index 46
Now on index 47
Now on index 48
Now on index 49
Now on index 50
Now on index 51
Now on index 52
Now on index 53
Now on index 54
Now on index 55
Now on index 56
Now on index 57
Now on index 58
Now on index 59
Now on index 60
Now on index 61
Now on index 62
No

In [51]:
df['coordinate'] = None
df.head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
0,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG FLAMBOYAN,None
1,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H MASRIKI,None
2,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAEMIN,None
3,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H SAIMIN,None
4,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG H. SAIMIN,None


In [52]:
for i in dict:
    df['coordinate'].iloc[i] = dict[i]

In [53]:
df[ ~df['coordinate'].isna() ].head()

,provinsi,kota,kecamatan,kelurahan,nama_jalan,coordinate
6,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG JATI INDAH,"[-6.2599631, 106.773803]"
7,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,GG MASRIKI,"[-6.2626431, 106.7741125]"
19,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,H SAIMAN,"[-6.2614855, 106.7741336]"
22,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JATI INDAH,"[-6.2599631, 106.773803]"
23,DKI JAKARTA,JAKARTA SELATAN,KEBAYORAN LAMA,PONDOK PINANG,JL BANK EXIM,"[-6.263339, 106.7745935]"


In [54]:
df.to_csv('jakarta_street_coordinate.csv')